In [2]:
import sys
from pyspark.sql import SparkSession
from operator import add

import re

def createNameMap(nameList):
    nameMap = {}
    for name in nameList:
        nameMap[name[0]] = name[1]
    return nameMap

def append(x, y):
    for item in x:
        y.append(item)
    return y
    
def mapreduce(inPath, outPath, spark):
    lines  = spark.read.text(inPath) \
            .rdd.map(lambda x:x[0]) \
            .map(lambda x:x.split(',')).map(lambda x:[x[0], [x[1], int(x[2])]])
    summary = lines.map(lambda x: [x[0], x[1][1]]).reduceByKey(add)
    nameMap = createNameMap(summary.collect())
    
    weight = lines.map(lambda x: [x[0], [[x[1][0], x[1][1]/float(nameMap[x[0]])]]]) \
            .reduceByKey(append) #将同一人物的相关人物放到一起
    output = weight.sortByKey(ascending=True).collect()
    
    fout = open(outPath, 'w', encoding='utf-8')
    for (key, valueList) in output:
        fout.write(key + '\t')
        for value in valueList:
            fout.write(value[0] + ',' + str(value[1]) + '|')
        fout.write('\n')
    fout.close()
    
def drive():
    spark = SparkSession\
    .builder\
    .appName("Task3")\
    .getOrCreate()
    
    inPath = './../result/task2.out'
    outPath = './../result/task3.out'
    mapreduce(inPath, outPath, spark)
    print('Done.')
drive()

Done.
